In [1]:
!pip install pandas

In [3]:
import pandas as pd

In [4]:
intakes = pd.read_csv('data/aac_intakes.csv')

In [12]:
intakes.head(1)

,age_upon_intake,animal_id,animal_type,breed,color,datetime,datetime2,found_location,intake_condition,intake_type,name,sex_upon_intake
0,8 years,A706918,Dog,English Springer Spaniel,White/Liver,2015-07-05T12:59:00.000,2015-07-05T12:59:00.000,9409 Bluegrass Dr in Austin (TX),Normal,Stray,Belle,Spayed Female


In [5]:
outcomes = pd.read_csv('data/aac_outcomes.csv')

In [15]:
outcomes.head(1)

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male


In [17]:
intakes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80187 entries, 0 to 80186
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   age_upon_intake   80187 non-null  object
 1   animal_id         80187 non-null  object
 2   animal_type       80187 non-null  object
 3   breed             80187 non-null  object
 4   color             80187 non-null  object
 5   datetime          80187 non-null  object
 6   datetime2         80187 non-null  object
 7   found_location    80187 non-null  object
 8   intake_condition  80187 non-null  object
 9   intake_type       80187 non-null  object
 10  name              55603 non-null  object
 11  sex_upon_intake   80186 non-null  object
dtypes: object(12)
memory usage: 7.3+ MB


In [18]:
outcomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80681 entries, 0 to 80680
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   age_upon_outcome  80673 non-null  object
 1   animal_id         80681 non-null  object
 2   animal_type       80681 non-null  object
 3   breed             80681 non-null  object
 4   color             80681 non-null  object
 5   date_of_birth     80681 non-null  object
 6   datetime          80681 non-null  object
 7   monthyear         80681 non-null  object
 8   name              56116 non-null  object
 9   outcome_subtype   36893 non-null  object
 10  outcome_type      80667 non-null  object
 11  sex_upon_outcome  80679 non-null  object
dtypes: object(12)
memory usage: 7.4+ MB


**1) Top Five Places Where Animals Are Found**

In [14]:
intakes.groupby('found_location').size().sort_values(ascending=False)[:5]

found_location
Austin (TX)                          14443
Outside Jurisdiction                   948
Travis (TX)                            921
7201 Levander Loop in Austin (TX)      517
Del Valle (TX)                         411
dtype: int64

I got this answer by grouping the values by the 'found_location' variable, counting the number of rows for each grouping, sorting the counts in descending order and then finding the first five results.

The answer is Austin (TX), Outside Jurisdiction, Travis(TX), 7201 Levander Loop in Austin (TX), and Del Valle (TX)

**2) What is the average number of pets found in a month in the year 2015? Are there months where there is a higher number of animals found?**

In [17]:
intakes['year_found'] = pd.DatetimeIndex(intakes['datetime']).year

In [18]:
intakes['month_found'] = pd.DatetimeIndex(intakes['datetime']).month

In [41]:
intakes_by_month = intakes.loc[intakes['year_found']==2015].groupby('month_found').size()

In [37]:
intakes_by_month.mean()

1559.3333333333333

In [43]:
intakes_by_month > 1560

month_found
1     False
2     False
3     False
4     False
5      True
6      True
7      True
8      True
9      True
10     True
11    False
12    False
dtype: bool

**The mean number of intakes per month in 2015 is 1559.3333333333333 or 1560 pets. The months where the number of intakes were greater than this average were May to October 2015. I found these answers by creating a new data frame called intakes_by_month that had the number of intakes per month in 2015. I then found the average of these counts using the mean() function and then used a comparison operator to figure out which months had counts that were greater than 1560.** 

**3) Ratio of incoming pets versus adopted pets** 

In [24]:
len(intakes)

80187

This helps you describe the unique values in a certain column

In [10]:
outcomes.outcome_type.unique()


array(['Transfer', 'Adoption', 'Euthanasia', 'Return to Owner', 'Died',
       'Disposal', 'Relocate', 'Missing', nan, 'Rto-Adopt'], dtype=object)

In [18]:
adopted_pets = outcomes.loc[outcomes['outcome_type']=='Adoption']

In [21]:
len(adopted_pets)

34232

In [25]:
80187/34232

2.3424573498480954

**The ratio of incoming pets to adopted pets is 2.34. I calculated this by first finding the number of pets in the incoming database and dividing that number by the number of pets from the outcomes database that were classified as having an outcome type - 'Adoption'. Then I divided these two numbers to find the ratio.**

**4) Type of Each Animal in the Shelter** This is the count of each type of animal in the shelter.

In [6]:
intakes.groupby('animal_type').size()

animal_type
Bird           342
Cat          29659
Dog          45743
Livestock        9
Other         4434
dtype: int64

The answer above shows the number of each type of animal in the shelter. I found this by using the pandas size()and groupby() functions. 

**5)What are the adoption rates for specific breeds?
Find the top 5 dog breeds in the shelter (based on count) and then find the adoption percentage of each breed.**

In [13]:
breeds = intakes.loc[intakes['animal_type']=='Dog'].groupby('breed').size().sort_values(ascending=False)[:5]

In [14]:
breeds

breed
Pit Bull Mix                 6382
Chihuahua Shorthair Mix      4860
Labrador Retriever Mix       4841
German Shepherd Mix          1963
Australian Cattle Dog Mix    1105
dtype: int64

In [21]:
adopted_pets['breed'].count()

34232

In [22]:

# Calculate Percentage
breeds['adoption_percent'] = (breeds[1] / 
                      adopted_pets['breed'].count()) * 100
# Show the dataframe
breeds

breed
Pit Bull Mix                 6382.000000
Chihuahua Shorthair Mix      4860.000000
Labrador Retriever Mix       4841.000000
German Shepherd Mix          1963.000000
Australian Cattle Dog Mix    1105.000000
adoption_percent             7838.709677
dtype: float64

In [25]:
breeds[0]


6382.0